In [1]:
import serial
import sys
from time import time

In [2]:
ser = serial.Serial('COM4', 9600)
data = ser.readline()[:-2]
print(data)

b'= 0'


In [ ]:
from __future__ import print_function
import serial, time, io, datetime
from serial import Serial

addr = "COM4" ## serial port to read data from
baud = 9600 ## baud rate for instrument

ser = serial.Serial(
    port = addr,\
    baudrate = baud,\
    parity=serial.PARITY_NONE,\
    stopbits=serial.STOPBITS_ONE,\
    bytesize=serial.EIGHTBITS,\
    timeout=0)


print("Connected to: " + ser.portstr)

filename="myFile.txt"
datafile=open(filename, 'a')

## this will store each line of data
seq = []
count = 1 ## row index

while True:
    for i in ser.readline()[:-2]:
        seq.append(i) ## convert from ACSII?
#         joined_seq = ''.join(str(v) for v in seq) ## Make a string from array
        print(seq)

        if i == '\n':
            datafile.write("Line: " + str(count) + "" + str(datetime.datetime.now()) + joined_seq) ## append a timestamp to each row of data
            seq = []
            count += 1
            break
datafile.close()
ser.close()

In [ ]:
ser = serial.Serial('COM4', 9600)
seq = []
while True:
     for i in ser.readline()[:-2]:
#         seq.append(i) ## convert from ACSII?
#         joined_seq = ''.join(str(v) for v in seq) ## Make a string from array
        print(i)

In [ ]:
# with serial.Serial('COM4', 9600) as ser:
ser = serial.Serial('COM4', 9600)
#     x = ser.read(80) 
#     print(x)
# line = ser.readline()[:-2]
print(ser)
# flush any junk left in the serial buffer
# ser.flushInput()
# # ser.reset_input_buffer() # for pyserial 3.0+
# run = True

In [ ]:
start_time = time()
timepoints = []
ydata = []
while True:
    data = ser.readline()[:-2] #the last bit gets rid of the new-line chars
        # sometimes the incoming data is garbage, so just 'try' to do this
        # store the entire dataset for later
    ydata.append(data)
    timepoints.append(time()-start_time)
    current_time = timepoints[-1]
    print(data.type)


In [ ]:
print(ydata[0])

In [ ]:
%matplotlib notebook

# run this in a Jupyter (IPython) Notebook!
# modified from http://www.lebsanft.org/?p=48
# http://pyserial.readthedocs.org/en/latest/pyserial_api.html
import serial
import numpy as np
from matplotlib import pyplot as plt
from time import time

# If you're not using Linux, you'll need to change this
# check the Arduino IDE to see what serial port it's attached to
ser = serial.Serial('COM4', 9600)

# set plot to animated
plt.ion() 

start_time = time()
timepoints = []
ydata = []
yrange = [-0.1,5.1]
view_time = 4 # seconds of data to view at once
duration = 24 # total seconds to collect data

fig1 = plt.figure()
# http://matplotlib.org/users/text_props.html
fig1.suptitle('live updated data', fontsize='18', fontweight='bold')
plt.xlabel('time, seconds', fontsize='14', fontstyle='italic')
plt.ylabel('potential, volts', fontsize='14', fontstyle='italic')
plt.axes().grid(True)
line1, = plt.plot(ydata,marker='o',markersize=4,linestyle='none',markerfacecolor='red')
plt.ylim(yrange)
plt.xlim([0,view_time])

# flush any junk left in the serial buffer
ser.flushInput()
# ser.reset_input_buffer() # for pyserial 3.0+
run = True

# collect the data and plot a moving frame
while run:
    ser.reset_input_buffer()
    data = ser.readline()[:-2]
    
    # sometimes the incoming data is garbage, so just 'try' to do this
    try:
        # store the entire dataset for later
        ydata.append(float(data[0])*5.0/1024)
        timepoints.append(time()-start_time)
        current_time = timepoints[-1]
        
        # update the plotted data
        line1.set_xdata(timepoints)
        line1.set_ydata(ydata)
        
        # slide the viewing frame along
        if current_time > view_time:
            plt.xlim([current_time-view_time,current_time])
            
        # when time's up, kill the collect+plot loop
        if timepoints[-1] > duration: run=False
    
    # if the try statement throws an error, just do nothing
    except: pass
    
    # update the plot
    fig1.canvas.draw()

# plot all of the data you collected
fig2 = plt.figure()
# http://matplotlib.org/users/text_props.html
fig2.suptitle('complete data trace', fontsize='18', fontweight='bold')
plt.xlabel('time, seconds', fontsize='14', fontstyle='italic')
plt.ylabel('potential, volts', fontsize='14', fontstyle='italic')
plt.axes().grid(True)

plt.plot(timepoints, ydata,marker='o',markersize=4,linestyle='none',markerfacecolor='red')
plt.ylim(yrange)
fig2.show()

ser.close()